In [ ]:
import os,sys
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import scipy.stats as sc
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.models import load_model
from argparse import ArgumentParser

tf.keras.utils.set_random_seed(42)  # sets seeds for base-python, numpy and tf
tf.config.experimental.enable_op_determinism()

In [ ]:
dataset = "Q1744"
feature_type = "defdif"

In [ ]:
#original thermonet like
#training data
training_features_dir_dir = f"path_to_features/{dataset}_features/{dataset}_{feature_type}_direct/"
training_features_dir_rev = f"path_to_features/{dataset}_features/{dataset}_{feature_type}_reverse/"
training_dataset_path = f"path_to_datasets/datasets/{dataset}_direct.csv"
AUGMENT_REVERSE_MUTATIONS = True

#evaluation data
evaluation_features_dir_dir = f"path_to_features/Ssym_{feature_type}_direct/"
evaluation_features_dir_rev = f"path_to_features/Ssym_{feature_type}_reverse/"
evaluation_dataset_path = "path_to_datasets/Ssym.csv"

#paths to save models and pictures
#mp = f"/home/nata/work/Projects/Protein_stability_prediction/Thermonet_var/14656_Unique_Mutations_Voxel_Features_PDBs/Models_default_TH/TH_{feature_type}_{dataset}/"
#picspathsave = f"/home/nata/work/Projects/Protein_stability_prediction/Thermonet_var/14656_Unique_Mutations_Voxel_Features_PDBs/Models_default_TH/pics_loss_TH_{feature_type}_{dataset}/"
#evalpathsave = f"/home/nata/work/Projects/Protein_stability_prediction/Thermonet_var/14656_Unique_Mutations_Voxel_Features_PDBs/Models_default_TH/evaluation_TH_{feature_type}_{dataset}/"

#for f in [mp, picspathsave, evalpathsave]:
#    if os.path.exists(f) == False:
#        os.mkdir(f)

In [ ]:
#oriented thermonet like
#training data
training_features_dir_dir = f"path_to_oriented_features/{dataset}_{feature_type}_direct/"
training_features_dir_rev = f"path_to_oriented_features/{dataset}_{feature_type}_reverse/"
training_dataset_path = f"path_to_datasets/{dataset}_direct.csv"
AUGMENT_REVERSE_MUTATIONS = True

#evaluation data
evaluation_features_dir_dir = f"path_to_oriented_features/Ssym_oriented_features/Ssym_{feature_type}_direct/"
evaluation_features_dir_rev = f"path_to_oriented_features/Ssym_oriented_features/Ssym_{feature_type}_reverse/"
evaluation_dataset_path = "path_to_datasets/Ssym.csv"

#paths to save models and pictures
mp = f"path_to_save_models/TH_{feature_type}_{dataset}/"
picspathsave = f"path_to_save_pictures/pics_loss_TH_{feature_type}_{dataset}/"
evalpathsave = f"path_to_save_csvs/evaluation_TH_{feature_type}_{dataset}/"

for f in [mp, picspathsave, evalpathsave]:
    if os.path.exists(f) == False:
        os.mkdir(f)

In [ ]:
def load_data_training_set():
    
    print("1. Loading csv datasets")
    df = pd.read_csv(training_dataset_path)
    print(f'Total unique mutations: {len(df)}')

    #load direct features
    df['features'] = df.apply(lambda r: f'{training_features_dir_dir}/{r.pdb_id}/{r.pdb_id}_{r.wild_type}{r.position}{r.mutant}.npy', axis=1)
    df = df[df.features.apply(lambda v: os.path.exists(v))]
    print(f'Total mutations with features: {len(df)}')
    df.features = [np.load(f) for f in tqdm(df.features, desc="2. Loading features")]
    print(f'Total mutations after filtering: {len(df)}')
    df_train = df
    
    if AUGMENT_REVERSE_MUTATIONS:
        
        print('Augmenting reverse mutations')
        df_rev = pd.read_csv(training_dataset_path)
        df_rev.ddg = -df_rev.ddg

        
        df_rev['features'] = df_rev.apply(lambda r: f'{training_features_dir_rev}/{r.pdb_id}/{r.pdb_id}_{r.wild_type}{r.position}{r.mutant}.npy', axis=1)
        df_rev = df_rev[df_rev.features.apply(lambda v: os.path.exists(v))]
        print(f'Total mutations with features: {len(df)}')
        
        
        df_rev.features = [np.load(f) for f in tqdm(df_rev.features, desc="3. Loading features")]
        print(f'Total mutations after filtering: {len(df_rev)}')
        df_train = pd.concat([df_train, df_rev], axis=0).sample(frac=1.).reset_index(drop=True)

    df_train.features = df_train.features.apply(lambda k: np.transpose(k, (1, 2, 3, 0)))
    
    return df_train

def load_data_ssym_dir():
    
    print("Loading Ssym direct mutations")
    df = pd.read_csv(evaluation_dataset_path)
    print(f'Total unique mutations: {len(df)}')

    df['features'] = df.apply(lambda r: f'{evaluation_features_dir_dir}/{r.pdb_id}/{r.pdb_id}_{r.wild_type}{r.position}{r.mutant}.npy', axis=1)
    df = df[df.features.apply(lambda v: os.path.exists(v))]
    print(f'Total mutations with features: {len(df)}')
    df.features = [np.load(f) for f in tqdm(df.features, desc="2. Loading features")]
    print(f'Total mutations after filtering: {len(df)}')

    df_train = df
    df_train.features = df_train.features.apply(lambda k: np.transpose(k, (1, 2, 3, 0)))
    
    return df_train

def load_data_ssym_rev():
    
    print('Loading Ssym reverse mutations')
    df_rev = pd.read_csv(evaluation_dataset_path)
    df_rev.ddg = -df_rev.ddg

        
    df_rev['features'] = df_rev.apply(lambda r: f'{evaluation_features_dir_rev}/{r.pdb_id}/{r.pdb_id}_{r.wild_type}{r.position}{r.mutant}.npy', axis=1)
    df_rev = df_rev[df_rev.features.apply(lambda v: os.path.exists(v))]
    print(f'Total mutations with features: {len(df_rev)}')
        
        
    df_rev.features = [np.load(f) for f in tqdm(df_rev.features, desc="3. Loading features")]
    print(f'Total mutations after filtering: {len(df_rev)}')
    
    df_rev.features = df_rev.features.apply(lambda k: np.transpose(k, (1, 2, 3, 0)))
    
    return df_rev

def build_model(model_type='regression', conv_layer_sizes=(16, 16, 16), dense_layer_size=16, dropout_rate=0.25):
    """
    """
    # make sure requested model type is valid
    if model_type not in ['regression', 'classification']:
        print('Requested model type {0} is invalid'.format(model_type))
        sys.exit(1)
        
    # instantiate a 3D convnet
    model = models.Sequential()
    model.add(layers.Conv3D(filters=conv_layer_sizes[0], kernel_size=(3, 3, 3), input_shape=(16, 16, 16, 14)))
    model.add(layers.Activation(activation='relu'))
    model.add(layers.Conv3D(filters=conv_layer_sizes[1], kernel_size=(3, 3, 3)))
    model.add(layers.Activation(activation='relu'))
    model.add(layers.Conv3D(filters=conv_layer_sizes[2], kernel_size=(3, 3, 3)))
    model.add(layers.Activation(activation='relu'))
    model.add(layers.MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dropout(rate=dropout_rate))
    model.add(layers.Dense(units=dense_layer_size, activation='relu'))
    model.add(layers.Dropout(rate=dropout_rate))
    
    # the last layer is dependent on model type
    if model_type == 'regression':
        model.add(layers.Dense(units=1))
    else:
        model.add(layers.Dense(units=3, activation='softmax'))
        model.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=0.0001),
                      metrics=['accuracy'])
    
    return model

def rmse(y_val_direct, y_pred):

    rmse = tf.sqrt(tf.reduce_mean(tf.square(tf.squeeze(y_val_direct) - tf.squeeze(y_pred))))
    
    return rmse

def pearson_r(y_val_direct, y_pred):

    if tf.shape(y_val_direct)[0] == 1:
        y_val_direct = tf.concat([y_val_direct, y_val_direct], axis=0)
        y_pred = tf.concat([y_pred, y_pred], axis=0)

        pr, _ = tf.py_function(sc.pearsonr, [y_val_direct, y_pred], [tf.float64, tf.float64])
        #tf.print("Pearson correlation coefficient:", pr)
    else:
        y_val_direct = tf.squeeze(y_val_direct)
        y_pred = tf.squeeze(y_pred)
    
        pr, _ = tf.py_function(sc.pearsonr, [y_val_direct, y_pred], [tf.float64, tf.float64])
        #tf.print("Pearson correlation coefficient:", pr)

    return pr

class EvaluateAndStoreMetrics(Callback):
    def __init__(self, X_val, y_val, key_prefix):
        super().__init__()
        self.X_val = X_val
        self.y_val = y_val
        self.key_prefix = key_prefix

    def on_epoch_end(self, epoch, logs=None):
        # Evaluate the model on the validation set
        val_loss, val_mae, val_mse, val_rmse, val_pearson_r = self.model.evaluate(
            self.X_val, self.y_val, verbose=0
        )

        # Store the evaluation metrics in history.history
        key_loss = self.key_prefix + 'loss'
        key_mae = self.key_prefix + 'mae'
        key_mse = self.key_prefix + 'mse'
        key_rmse = self.key_prefix + 'rmse'
        key_pearson_r = self.key_prefix + 'pearson_r'

        logs[key_loss] = val_loss
        logs[key_mae] = val_mae
        logs[key_mse] = val_mse
        logs[key_rmse] = val_rmse
        logs[key_pearson_r] = val_pearson_r

        # Print or log the metrics if needed
        print(f"\nValidation Metrics after Epoch {epoch + 1}:")
        print(f" - {key_loss}: {val_loss:.4f}")
        print(f" - {key_mae}: {val_mae:.4f}")
        print(f" - {key_mse}: {val_mse:.4f}")
        print(f" - {key_rmse}: {val_rmse:.4f}")
        print(f" - {key_pearson_r}: {val_pearson_r:.4f}")

In [ ]:
df_train = load_data_training_set()
df_train_ssym_dir = load_data_ssym_dir()
df_train_ssym_rev = load_data_ssym_rev()

In [ ]:
# instantiate the model
conv_layer_sizes = (16, 24, 32)
dense_layer_size = 24
model = build_model('regression', conv_layer_sizes, dense_layer_size)
model.summary()

In [ ]:
tf.keras.utils.set_random_seed(42)  # sets seeds for base-python, numpy and tf
tf.config.experimental.enable_op_determinism()
X_direct = df_train.features.to_list()
X_direct = np.array(X_direct)
y_direct = df_train.ddg.to_numpy()
sample_size = X_direct.shape[0]
k = 10
num_validation_samples = sample_size // k
shp = df_train.shape[0]//10


In [ ]:
results = []
ev = []
log_loss_fold = []


fold_num=0
for i in (range(k)):
    # create a validation set
    X_val_direct = X_direct[i * num_validation_samples: (i + 1) * num_validation_samples]
    y_val_direct = y_direct[i * num_validation_samples: (i + 1) * num_validation_samples]

    # create the training data from all other partitions
    X_train_direct = np.concatenate(
        [X_direct[:i * num_validation_samples],
         X_direct[(i + 1) * num_validation_samples:]],
        axis=0
    )
    y_train_direct = np.concatenate(
        [y_direct[:i * num_validation_samples],
         y_direct[(i + 1) * num_validation_samples:]],
        axis=0
    )

    # shuffle the training set
    indices = np.arange(0, X_train_direct.shape[0])
    np.random.shuffle(indices)
    X_train = X_train_direct[indices]
    y_train = y_train_direct[indices]


    #prepare validation sets on Ssym direct dataset
    X_direct_ssym_dir = np.array(df_train_ssym_dir.features.to_list())
    y_direct_ssym_dir = df_train_ssym_dir.ddg.to_numpy()


    #prepare validation sets on Ssym reverse dataset
    X_direct_ssym_rev = np.array(df_train_ssym_rev.features.to_list())
    y_direct_ssym_rev = df_train_ssym_rev.ddg.to_numpy()

    
    # checkpoint
    model_path = mp+"kerv1" + '_member_' + str(i + 1) + '.h5'
    m_nm = "kerv1" + '_member_' + str(i + 1) + '.h5'
    checkpoint = callbacks.ModelCheckpoint(model_path, monitor='val_loss', verbose=1,
                                           save_best_only=True, mode='min')
    
    early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10)
    evaluate_callback_Ssym_dir = EvaluateAndStoreMetrics(X_direct_ssym_dir, y_direct_ssym_dir, key_prefix='val_direct_')

    evaluate_callback_Ssym_rev = EvaluateAndStoreMetrics(X_direct_ssym_rev, y_direct_ssym_rev, key_prefix='val_rev_')
    
    callbacks_list = [checkpoint, early_stopping, evaluate_callback_Ssym_dir, evaluate_callback_Ssym_rev]

    # build a model
    model = build_model(model_type='regression', conv_layer_sizes=conv_layer_sizes,
                        dense_layer_size=dense_layer_size, dropout_rate=0.5)
    
    model.compile(loss='mse', optimizer=optimizers.Adam(
        learning_rate=0.001,
        beta_1=0.9,
        beta_2=0.999,
        amsgrad=False
        ),
        metrics=['mae', 'mse', rmse, pearson_r]
                 )

    # fit the model
    history = model.fit(X_train_direct, y_train_direct, validation_data=(X_val_direct, y_val_direct),
                        epochs=100, batch_size=8, callbacks=callbacks_list, verbose=1)
    result = {
        'fold': fold_num,
        'val_loss': min(history.history['val_loss']),
        'val_mae': min(history.history['val_mae']),
        'val_mse': min(history.history['val_mse']),
        'val_rmse': min(history.history['val_rmse']),
        'val_pearson_r': max(history.history['val_pearson_r'])
    }
    
    results.append(result)
    log_loss_fold.append(min(history.history['val_loss']))
    fold_num += 1
        
    ev.append({'min_loss_CV': min(log_loss_fold), 'mean_loss_CV': np.mean(log_loss_fold)})
    
    df_results_inter = pd.DataFrame(results)
    ev_results_inter =pd.DataFrame(ev)
    ev_results_inter.to_csv(evalpathsave, index=False)
    df_results_inter.to_csv(evalpathsave, index=False)
    
    
    # Access the training history
    epochs = range(1, len(history.history['loss']) + 1)
    
    plt.figure(figsize=(12, 6))

    # Subplot 1: Training and Validation Losses
    plt.subplot(1, 2, 1)
    plt.plot(epochs, history.history['loss'], label='Training Loss')
    plt.plot(epochs, history.history['val_loss'], label='Validation Loss')
    plt.title(f'{m_nm} Training and Validation Losses')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # Subplot 2: MSE, RMSE and Pearson_r
    plt.subplot(1, 2, 2)
    plt.plot(epochs, history.history['mse'], label='Training MSE')
    plt.plot(epochs, history.history['rmse'], label='Training RMSE')
    plt.plot(epochs, history.history['pearson_r'], label='Training Pearson_r')
    plt.plot(epochs, history.history['val_mse'], label='Validation MSE')
    plt.plot(epochs, history.history['val_rmse'], label='Validation RMSE')
    plt.plot(epochs, history.history['val_pearson_r'], label='Validation Pearson_r')
    plt.title(f'{m_nm} MSE, RMSE and Pearson_r on Training and Validation')
    plt.xlabel('Epochs')
    plt.ylabel('Metric')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig(f"{picspathsave}/{m_nm}_{feature_type}_losses_def.png", dpi = 300)
    plt.show()

    # Plotting evaluation metrics for the second validation set
    plt.figure(figsize=(8, 6))

    plt.plot(epochs, history.history['val_direct_loss'], label='Validation Loss Ssym_dir')
    plt.plot(epochs, history.history['val_direct_mae'], label='Validation MAE Ssym_dir')
    plt.plot(epochs, history.history['val_direct_mse'], label='Validation MSE Ssym_dir')
    plt.plot(epochs, history.history['val_direct_rmse'], label='Validation RMSE Ssym_dir')
    plt.plot(epochs, history.history['val_direct_pearson_r'], label='Validation Pearson_r Ssym_dir')

    plt.title(f'{m_nm} Evaluation Metrics for Ssym_dir')
    plt.xlabel('Epochs')
    plt.ylabel('Metric')
    plt.legend()
    plt.savefig(f"{picspathsave}/{m_nm}_{feature_type}_ssym_dir_def.png", dpi = 300)
    plt.show()

    # Plotting evaluation metrics for the third validation set
    plt.figure(figsize=(8, 6))

    plt.plot(epochs, history.history['val_rev_loss'], label='Validation Loss Ssym_rev')
    plt.plot(epochs, history.history['val_rev_mae'], label='Validation MAE Ssym_rev')
    plt.plot(epochs, history.history['val_rev_mse'], label='Validation MSE Ssym_rev')
    plt.plot(epochs, history.history['val_rev_rmse'], label='Validation RMSE Ssym_rev')
    plt.plot(epochs, history.history['val_rev_pearson_r'], label='Validation Pearson_r Ssym_rev')

    plt.title(f'{m_nm} Evaluation Metrics for Ssym_rev')
    plt.xlabel('Epochs')
    plt.ylabel('Metric')
    plt.legend()
    plt.savefig(f"{picspathsave}/{m_nm}_{feature_type}_ssym_rev_def.png", dpi = 300)
    plt.show()